<h1 style="font-family:Audrey;" "font-size:20px;" align="center" ><b> Fake news detection </b></h1>

<h2 style="font-family:Audrey;" "font-size:20px;" align="center" ><b> Author </b></h2>

- Name:- Mohsin Shareef <br>
- [Lets Connect on LinkedIn](www.linkedin.com/in/mohsin-shareef-9231a1294)
- [Lets Connect on GitHub](https://github.com/MohsinShareef)
- [Lets Connect on Kaggle](https://www.kaggle.com/mohsin31202)

<h2 style="font-family:Audrey;" "font-size:15px;" align="center" > <b>About Data </b></h2>
<p style="padding: 10px; text-color:white;" "font-family:Audrey;">

                ID: Unique id for a news article.

                Title: The title of a news article.

                Author: Author of the news article.

                Text: The text of the article; could be incomplete.

                Label: A label that marks whether the news article is real or fake:-

                1: Fake news.
                0: Real News.
</p>

- Import important libraries

In [4]:
# import libraries for data manipulation 
import pandas as pd
import numpy as np

# import libraries Natural Language Toolkit 
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

# import machine learning libraries
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package stopwords to C:\Users\1st Choice
[nltk_data]     Computer\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<h1 style="font-family:Audrey;" "font-size:15px;" align="center" > <b>Load the Dataset </b></h1>

In [5]:
# read the dataset
df = pd.read_csv('data/news_data.csv')
# show the first 5 rows
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


<h2 style="font-family:Audrey;" "font-size:15px;" align="center" > <b>Basic Information of Datset </b></h2>

In [6]:
# information about the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


<h2 style="font-family:Audrey;" "font-size:15px;" align="center" > <b>Shape of the Datset </b></h2>

In [7]:
# check the shape of the dataset
df.shape

(20800, 5)

<h1 style="font-family:Audrey;" "font-size:15px;" align="center" > <b>Handle the Missing values </b></h1>

In [8]:
# check the null values
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [9]:
# fill the null values
df.fillna('', inplace=True)
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

<h2 style="font-family:Audrey;" "font-size:15px;" align="center" > <b>Check the Dublicate Values</b></h2>

In [10]:
# Check the dublicate values
df.duplicated().sum()

0

<h1 style="font-family:Audrey;" "font-size:15px;" align="center" > <b>Feature Engineering</b></h1>

In [11]:
# Merge the 'title' and 'author' column
df['content'] = df['title'] + ' ' + df['author']
df.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"FLYNN: Hillary Clinton, Big Woman on Campus - ..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Why the Truth Might Get You Fired Consortiumne...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,15 Civilians Killed In Single US Airstrike Hav...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Iranian woman jailed for fictional unpublished...


<h2 style="font-family:Audrey;" "font-size:15px;" align="center" > <b> Clean the data for Machine Learning</b></h2>

In [12]:
# create a function for stemming
ps = PorterStemmer()
def stemming(content):
    stemming_content = re.sub('[^a-zA-Z]',' ',content)
    stemming_content = stemming_content.lower()
    stemming_content = stemming_content.split()
    stemming_content = [ps.stem(word) for word in stemming_content if not word in stopwords.words('english')]
    stemming_content = ' '.join(stemming_content)
    return stemming_content

In [13]:
# apply the function
df['content'] = df['content'].apply(stemming)

In [14]:
# Check the content column
df['content']

0        hous dem aid even see comey letter jason chaff...
1        flynn hillari clinton big woman campu breitbar...
2                   truth might get fire consortiumnew com
3        civilian kill singl us airstrik identifi jessi...
4        iranian woman jail fiction unpublish stori wom...
                               ...                        
20795    rapper trump poster child white supremaci jero...
20796    n f l playoff schedul matchup odd new york tim...
20797    maci said receiv takeov approach hudson bay ne...
20798    nato russia hold parallel exercis balkan alex ...
20799                            keep f aliv david swanson
Name: content, Length: 20800, dtype: object

<h2 style="font-family:Audrey;" "font-size:15px;" align="center" > <b>Splitting the Dataset into X and y</b></h2>

In [15]:
# create X and y
X = df['content'].values
y = df['label'].values

In [16]:
# create a vector
vector = TfidfVectorizer()
vector.fit(X)
X = vector.transform(X)

In [17]:
print(X)

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420

<h2 style="font-family:Audrey;" "font-size:15px;" align="center" > <b>Splitting Dataset Into Training and Testing Set</b></h2>

In [18]:
# split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=2)

In [19]:
# check the shape
X_train.shape

(16640, 17128)

<h1 style="font-family:Audrey;" "font-size:15px;" align="center" > <b>Select the Best Model</b></h1>

In [20]:
# create a list of models
models = [
    ('Logistic Regression',LogisticRegression(random_state=42)),
    ('Random Forest Classifier', RandomForestClassifier(random_state=42)),
    ('BernoulliNB', BernoulliNB())
]

# train the models
model_scores = []
for name, model in models:
    model.fit(X_train,y_train)

    y_pred = model.predict(X_test)
    metric = accuracy_score(y_pred,y_test)
    model_scores.append((name,metric))
    sorted_models = sorted(model_scores, key=lambda x: x[1], reverse=False)
for model in sorted_models:
    print('Accuracy Score for', f"{model[0]} is {model[1]: .2f}")
    # print("Best Model is", f"{model[0]} and accuracy is {model[1].max()}")
    best_model = model[0],model[1].max()
    print("Best Models is ", best_model)

Accuracy Score for Logistic Regression is  0.98
Best Models is  ('Logistic Regression', 0.9790865384615385)
Accuracy Score for BernoulliNB is  0.98
Best Models is  ('BernoulliNB', 0.9829326923076923)
Accuracy Score for Random Forest Classifier is  0.99
Best Models is  ('Random Forest Classifier', 0.9935096153846154)


<h2 style="font-family:Audrey;" "font-size:15px;" align="center" > <b>Apply the Machine Learning Model</b></h2>

In [21]:
# create a best model
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train,y_train)

y_pred = rf.predict(X_test)
prediction= (f"{accuracy_score(y_pred,y_test)}")
print(f'Accuracy of Random Forest Model is {prediction}')

Accuracy of Random Forest Model is 0.9935096153846154


<h2 style="font-family:Audrey;" "font-size:15px;" align="center" > <b>Make the Prediction</b></h2>

In [22]:
# make a prediction
input_data = X_test[10]
result = rf.predict(input_data)

In [23]:
# print the result
if (result[0] == 0):
    print('The news is Real')
else:
    print('The news is Fake')

The news is Real
